# Pandas GroupBy: Grouping Data in Python

## Why Groupby?
## - Because the data is usually not as we need it
### Ex: analyse the COVID19 cases by state using this city table
#### (you cannot ask anybody to generate and send a state table to you! XD)

<img src = 'https://i.imgur.com/40sH0Th.png'>


## GroupBy works in 3 steps:
### - Splitting the data
### - Applying a function
### - Combining the results

----

# Table of contents:
0. Getting started:  
0.1. Downloading the files  
0.2. Setting up the environment
<br></br>
1. Analysing the US. Congress dataset: basic groupby functions  
1.1. How many women did the US congress have until today?
<br></br>
2. Analysing the air quality dataset: floats and time series  
2.1. Which year had the worst air quality? Why?  
<br></br>
3. Putting it all together
3.1. Which state had the most cases and deaths of COVID?

----

##  DISCLAIMER ##
#### This class was heavily based on https://realpython.com/pandas-groupby/
- Several contents from the original source were stripped of   
due to time constraint and complexity
----

# Part 0: getting ready
### - create a folder in your computer for this project
### - download and save the zip file on it
link: https://github.com/realpython/materials/tree/master/pandas-groupby?__s=y8yfy0b4k55hcke3amk7
### - unzip the file contents
### - import pandas

In [65]:
import pandas as pd

----
# Part 1: How many women did the US congress have until today?

In [66]:
dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}
df = pd.read_csv(
    "legislators-historical.csv",
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "last_name"],
    parse_dates=["birthday"]
)

In [67]:
df.tail()

,last_name,first_name,birthday,gender,type,state,party
11970,Garrett,Thomas,1972-03-27,M,rep,VA,Republican
11971,Handel,Karen,1962-04-18,F,rep,GA,Republican
11972,Jones,Brenda,1959-10-24,F,rep,MI,Democrat
11973,Marino,Tom,1952-08-15,M,rep,PA,Republican
11974,Jones,Walter,1943-02-10,M,rep,NC,Republican


In [68]:
df.groupby(["state", "gender"])["last_name"].count()

state  gender
AK     F           0
       M          16
AL     F           3
       M         203
AR     F           5
                ... 
WI     M         196
WV     F           1
       M         119
WY     F           2
       M          38
Name: last_name, Length: 116, dtype: int64

In [69]:
df.groupby("state", sort=False)["last_name"].count().head()

state
DE      97
VA     432
SC     251
MD     305
PA    1053
Name: last_name, dtype: int64

## Splitting the data: a closer look
### - Looking at all the states

In [70]:
by_state = df.groupby("state")
for state, frame in by_state:
     print(f"First 2 entries for {state!r}")
     print("------------------------")
     print(frame.head(2), end="\n\n")

First 2 entries for 'AK'
------------------------
     last_name first_name   birthday gender type state        party
6619    Waskey      Frank 1875-04-20      M  rep    AK     Democrat
6647      Cale     Thomas 1848-09-17      M  rep    AK  Independent

First 2 entries for 'AL'
------------------------
    last_name first_name   birthday gender type state       party
912   Crowell       John 1780-09-18      M  rep    AL  Republican
991    Walker       John 1783-08-12      M  sen    AL  Republican

First 2 entries for 'AR'
------------------------
     last_name first_name   birthday gender type state party
1001     Bates      James 1788-08-25      M  rep    AR   NaN
1279    Conway      Henry 1793-03-18      M  rep    AR   NaN

First 2 entries for 'AS'
------------------------
          last_name first_name   birthday gender type state     party
10797         Sunia       Fofó 1937-03-13      M  rep    AS  Democrat
11755  Faleomavaega        Eni 1943-08-15      M  rep    AS  Democrat

F

In [71]:
by_state.get_group("PA")

,last_name,first_name,birthday,gender,type,state,party
4,Clymer,George,1739-03-16,M,rep,PA,NaN
19,Maclay,William,1737-07-20,M,sen,PA,Anti-Administration
21,Morris,Robert,1734-01-20,M,sen,PA,Pro-Administration
27,Wynkoop,Henry,1737-03-02,M,rep,PA,NaN
38,Jacobs,Israel,1726-06-09,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11891,Brady,Robert,1945-04-07,M,rep,PA,Democrat
11932,Shuster,Bill,1961-01-10,M,rep,PA,Republican
11945,Rothfus,Keith,1962-04-25,M,rep,PA,Republican
11959,Costello,Ryan,1976-09-07,M,rep,PA,Republican


In [72]:
state, frame = next(iter(by_state)) 

In [73]:
state

'AK'

In [74]:
frame.head(3)

,last_name,first_name,birthday,gender,type,state,party
6619,Waskey,Frank,1875-04-20,M,rep,AK,Democrat
6647,Cale,Thomas,1848-09-17,M,rep,AK,Independent
7442,Grigsby,George,1874-12-02,M,rep,AK,NaN


### Warning! A wild challenge approaches! 
- So, how many women did the US congress have until today?

In [75]:
df.groupby('').count()

KeyError: ''

# Part 2: Which year had the worst air quality? Why?

In [76]:
df = pd.read_csv(
    "airqual.csv",
    parse_dates=[["Date", "Time"]],
    na_values=[-200],
    usecols=["Date", "Time", "CO(GT)", "T", "RH", "AH"]
).rename(
    columns={
        "CO(GT)": "co",
        "Date_Time": "tstamp",
        "T": "temp_c",
        "RH": "rel_hum",
        "AH": "abs_hum",
    }
).set_index("tstamp")

In [77]:
df.head()

,co,temp_c,rel_hum,abs_hum
tstamp,,,,
2004-03-10 18:00:00,2.6,13.6,48.9,0.7578
2004-03-10 19:00:00,2.0,13.3,47.7,0.7255
2004-03-10 20:00:00,2.2,11.9,54.0,0.7502
2004-03-10 21:00:00,2.2,11.0,60.0,0.7867
2004-03-10 22:00:00,1.6,11.2,59.6,0.7888


In [78]:
df.index.min()

Timestamp('2004-03-10 18:00:00')

In [79]:
df.index.min()

Timestamp('2004-03-10 18:00:00')

### Splitting the data using dates: a closer look
- Which was the weekday with the highest CO mean?

In [80]:
df.index.day_name()

Index(['Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       ...
       'Monday', 'Monday', 'Monday', 'Monday', 'Monday', 'Monday', 'Monday',
       'Monday', 'Monday', 'Monday'],
      dtype='object', name='tstamp', length=9357)

In [81]:
day_names = df.index.day_name()
df.groupby(day_names)["co"].mean()

tstamp
Friday       2.543041
Monday       2.016741
Saturday     1.861077
Sunday       1.438069
Thursday     2.455505
Tuesday      2.382267
Wednesday    2.400787
Name: co, dtype: float64

In [82]:
df.index.hour

Int64Index([18, 19, 20, 21, 22, 23,  0,  1,  2,  3,
            ...
             5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
           dtype='int64', name='tstamp', length=9357)

In [83]:
df.groupby([df.index.year, df.index.quarter])["co"].agg(["max", "min"])

max  min
tstamp tstamp           
2004   1        8.1  0.3
       2        7.3  0.1
       3        7.5  0.1
       4       11.9  0.1
2005   1        8.7  0.1
       2        5.0  0.3

In [84]:
df.resample("M")["co"].agg(["max", "min"]).rename_axis('quarter')

,max,min
quarter,,
2004-03-31,8.1,0.3
2004-04-30,7.3,0.3
2004-05-31,6.5,0.1
2004-06-30,6.4,0.1
2004-07-31,5.3,0.1
2004-08-31,3.5,0.1
2004-09-30,7.5,0.2
2004-10-31,9.5,0.4
2004-11-30,11.9,0.1


In [85]:
df.resample("Y")["co"].mean()

tstamp
2004-12-31    2.190572
2005-12-31    2.055447
Freq: A-DEC, Name: co, dtype: float64

# Part 3: putting it all together:



In [86]:
import requests

csv_file = requests.get('https://data.brasil.io/dataset/covid19/caso_full.csv.gz')
with open('covid_dados.csv.gz', 'wb') as x:
    x.write(csv_file.content)
df = pd.read_csv(
    'covid_dados.csv.gz',
    usecols = ['city','date','state','place_type','new_confirmed','new_deaths'],
    parse_dates = ['date']
)

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343354 entries, 0 to 343353
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   city           340110 non-null  object        
 1   date           343354 non-null  datetime64[ns]
 2   place_type     343354 non-null  object        
 3   state          343354 non-null  object        
 4   new_confirmed  343354 non-null  int64         
 5   new_deaths     343354 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 15.7+ MB


In [88]:
df.tail()

,city,date,place_type,state,new_confirmed,new_deaths
343349,NaN,2020-07-10,state,RS,1056,49
343350,NaN,2020-07-10,state,SC,1698,12
343351,NaN,2020-07-10,state,SE,1297,30
343352,NaN,2020-07-10,state,SP,9395,324
343353,NaN,2020-07-10,state,TO,0,0


## Remembering last class: how to select only states?

In [89]:
df.loc[df.place_type == 'state']

,city,date,place_type,state,new_confirmed,new_deaths
1,NaN,2020-02-25,state,SP,1,0
3,NaN,2020-02-26,state,SP,0,0
5,NaN,2020-02-27,state,SP,0,0
7,NaN,2020-02-28,state,SP,1,0
9,NaN,2020-02-29,state,SP,0,0
...,...,...,...,...,...,...
343349,NaN,2020-07-10,state,RS,1056,49
343350,NaN,2020-07-10,state,SC,1698,12
343351,NaN,2020-07-10,state,SE,1297,30
343352,NaN,2020-07-10,state,SP,9395,324


## How do we confirm that the number of new cases per state are equal to city cases?

In [90]:
df_states = df.loc[df.place_type == 'state']
df_cities = df.loc[df.place_type == 'city']

In [91]:
df_cities.groupby('state').new_confirmed.sum()

state
AC     15465
AL     43191
AM     82507
AP     31080
BA     98319
CE    131324
DF     65677
ES     61361
GO     35685
MA     95323
MG     70086
MS     11671
MT     24804
PA    124934
PB     57619
PE     68767
PI     31269
PR     37424
RJ    128324
RN     37120
RO     25402
RR     21220
RS     37490
SC     38408
SE     33416
SP    349715
TO     13845
Name: new_confirmed, dtype: int64

In [92]:
df_states.groupby('state').new_confirmed.sum()

state
AC     15768
AL     43959
AM     82507
AP     31080
BA    101186
CE    134610
DF     67297
ES     61361
GO     35706
MA     96718
MG     70086
MS     12261
MT     26396
PA    124934
PB     59118
PE     70100
PI     31269
PR     39512
RJ    129443
RN     38342
RO     26000
RR     21220
RS     37490
SC     40106
SE     34713
SP    359110
TO     14509
Name: new_confirmed, dtype: int64

In [93]:
by_city = df_cities.groupby('state').new_confirmed.sum()
by_state = df_states.groupby('state').new_confirmed.sum()

In [94]:
by_city.equals(by_state)

False

In [95]:
df_states.head()

,city,date,place_type,state,new_confirmed,new_deaths
1,NaN,2020-02-25,state,SP,1,0
3,NaN,2020-02-26,state,SP,0,0
5,NaN,2020-02-27,state,SP,0,0
7,NaN,2020-02-28,state,SP,1,0
9,NaN,2020-02-29,state,SP,0,0


## Warning! A wild challenger approaches!
### - Which state had the most cases and deaths?

In [96]:
df_states.groupby('_____')[['new_confirmed','new_deaths']].___()

KeyError: '_____'

# Warning! A terrible challenger approaches!
## - Can you tell me which was the month with the most cases in São Paulo?

In [105]:
df_states.set_index('').resample('')[['new_confirmed','new_deaths']].()

,new_confirmed,new_deaths
date,,
2020-02-29,2,0
2020-03-31,5822,202
2020-04-30,81302,5778
2020-05-31,428968,23387
2020-06-30,896570,30424
2020-07-31,392137,10775


# You have reached the end of this class. Congratulations!!!!
